In [ ]:
data = pd.read_table("imhonet-films.tsv")
data = data.dropna(how='any')
data = data.drop_duplicates(subset=['score', 'content_id', 'element_id', 'user_id'])

In [ ]:
user_id_list = data.user_id
item_id_list = data.element_id
rating_list = data.score
sf = graphlab.SFrame({ 'user_id': user_id_list,
                       'item_id': item_id_list,
                       'rating': rating_list})

In [ ]:
train, test = graphlab.recommender.util.random_split_by_user(sf, item_test_proportion=0.2)

In [ ]:
import graphlab.aggregate as agg
train_user_count = train.groupby(key_columns='user_id', operations={'count_of_marked_films': agg.COUNT()})
test_user_count = test.groupby(key_columns='user_id', operations={'count_of_marked_films': agg.COUNT()})

In [ ]:
train_film_count = np.array(train_user_count['count_of_marked_films'])
test_film_count = np.array(test_user_count['count_of_marked_films'])

In [ ]:
fig = plt.figure(figsize=(20, 5))

plt.subplot (1, 2, 1)
plt.hist(train_film_count, bins=100, log=True, range=[0, 250], color='g')
plt.title ("Train_User_Film")
plt.xlabel("Film Marked Count")
plt.ylabel("User Count")

plt.subplot (1, 2, 2)
plt.hist(test_film_count, bins=100, log=True, range=[0, 250], color='b')
plt.title ("Test_User_Film")
plt.xlabel("Film Marked Count")
plt.ylabel("User Count")

In [ ]:
sf = graphlab.SFrame(train)

m2 = graphlab.recommender.popularity_recommender.create(sf, target='rating')

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

def print_metrics(model, sf_test, name):
    
    df_test = sf_test.to_dataframe()
    all_user_id_test = np.unique(test['user_id'])
    list_mse = []
    list_SPR = []

    for user_id in all_user_id_test:
        tmp = df_test[df_test['user_id'] == user_id]
        user_id_list = list(tmp.user_id)
        item_id_list = list(tmp.item_id)

        if (len(item_id_list) <= 1):
            continue
        y_test = list(tmp.rating)
        sf_test = graphlab.SFrame({'user_id': user_id_list,
                                   'item_id': item_id_list})
        y_pred = model.predict(sf_test)    
        y_pred = list(y_pred)

        MSE = mean_squared_error(y_test, y_pred)
        list_mse.append(MSE)

        SPR = spearmanr(y_test, y_pred)  

        if (np.isnan(SPR[0])):
            y_test.append(1)
            y_pred.append(1)
            SPR = spearmanr(y_test, y_pred) 

        list_SPR.append(SPR[0])

    print name    
    print "MSE:", np.average(list_mse)
    print "Spearman:", np.average(list_SPR)
    print "MAP:", mean_average_precision(model, df_pair_user_film_test)

In [ ]:
print_metrics(m2,test,'')